<a href="https://colab.research.google.com/github/kamazoun/webscrapping-with-Python/blob/master/webscrapping_customs_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def process_details(details):
  try:
    ls = details.split("\n")
    fls = []
    for l in ls:
      l = l.strip()
      if l != "": fls.append(l)
    return fls[0], fls[1], fls[2]
  except:
    return "", "", details

def process_destination(dest):
  try:
    ls = dest.split("\n")
    fls = []
    for l in ls:
      l = l.strip()
      if l != "": fls.append(l)
    return fls[0], fls[1]
  except:
    return "", dest

In [0]:
export_list = []
import_list = []

def extract_exports(info):
  d  = {}
  for row in info:
    d["exporter"] = row.find_all("div", {"class": "col-xs-2"})[0].text.strip() #useless, exporter is company
    d["importer"] = row.find_all("div", {"class": "col-xs-2"})[1].text.strip()
    details = row.find("div", {"class": "col-xs-5"}).text.strip()
    details_ls = []
    d["detail 1"], d["detail 2"], d["detail 3"] = process_details(details) # a list
    d["Departure"] = row.find_all("div", {"class": "col-xs-6"})[0].text.strip()
    destination = row.find_all("div", {"class": "col-xs-6"})[1].text.strip()
    d["Arrival date"], d["Destination"] = process_destination(destination)
    
    export_list.append(d)
  

  
def extract_imports(info):
  d = {}
  for row in info:
    d["exporter"] = row.find_all("div", {"class": "col-xs-2"})[0].text.strip()
    d["importer"] = row.find_all("div", {"class": "col-xs-2"})[1].text.strip() #useless, exporter is company
    details = row.find("div", {"class": "col-xs-5"}).text.strip()
    details_ls = []
    d["detail 1"], d["detail 2"], d["detail 3"] = process_details(details) # a list
    d["Departure"] = row.find_all("div", {"class": "col-xs-6"})[0].text.strip()
    destination = row.find_all("div", {"class": "col-xs-6"})[1].text.strip()
    d["Arrival date"], d["Destination"] = process_destination(destination)
    
    import_list.append(d)


In [0]:
def urls_in_page(page):
  r = requests.get(page)
  c = r.content
  soup = BeautifulSoup(c, "html.parser")
  all_links = soup.find_all("ul", {"class": "directory-results directory-results-3column"})
  hrefs = []
  for links in all_links:
    try:
      l = links.find_all("a")
    except:
      return []
    for link in l:
      hrefs.append(link.get('href')) 
  return hrefs   



def link_pages():
  base_url = "http://www.tradesparq.com/company-directory/"
  page = []
  for i in range(0, 150):
    for j in range(0, 150):
      page.append(base_url + str(i + 1) + "/" + str(j + 1))
   
  return page

In [0]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

exports_data = []
#pages = link_pages()

counter = 0
for page in link_pages():
  #hrefs = urls_in_page(page)
  for href in urls_in_page(page):
    try:
      r = requests.get(href)
      c = r.content
      soup = BeautifulSoup(c, "html.parser")

      exports = soup.find_all("section", {"class": "shipments-section"})[0].find_all("div", {"class": "shipment-record row"})
      imports = soup.find_all("section", {"class": "shipments-section"})[1].find_all("div", {"class": "shipment-record row"})
    except:
      print("error at counter " + str(counter))
      print("\nPage : " + href)
      pass
    
    extract_exports(exports)
    extract_imports(imports)
  
    counter = counter + 1
  
  
df_export = pd.DataFrame(export_list)
df_import = pd.DataFrame(import_list)
#df
df_export.to_excel("export_output.xlsx")
df_import.to_excel("import_output.xlsx")